# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#import "clean cities csv"
file = "../output_data/clean_cities_data.csv"
weather_df = pd.read_csv(file)
weather_df

,City Name,Country Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
0,Severo-Kuril'sk,RU,50.68,156.12,51.37,83,100,5.08,2020-07-24 02:45:54
1,Lavrentiya,RU,65.58,-171.00,56.17,61,100,8.59,2020-07-24 02:49:45
2,San Patricio,MX,19.22,-104.70,82.40,74,75,4.83,2020-07-24 02:41:51
3,Barrow,US,71.29,-156.79,39.20,93,90,8.05,2020-07-24 02:45:25
4,Muisne,EC,0.60,-80.03,73.42,85,99,8.39,2020-07-24 02:49:45
...,...,...,...,...,...,...,...,...,...
575,Turukhansk,RU,65.82,87.98,56.21,70,100,7.96,2020-07-24 02:50:52
576,Sandpoint,US,48.28,-116.55,72.00,46,1,3.96,2020-07-24 02:50:11
577,Sancti Spíritus,CU,21.93,-79.44,74.57,91,100,3.44,2020-07-24 02:50:53
578,Ambilobe,MG,-13.20,49.05,68.68,61,78,1.61,2020-07-24 02:50:08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#access maps with unique API key
gmaps.configure(api_key = g_key)

In [6]:
#create coordinates
records = weather_df[["Latitude", "Longitude"]].to_records(index=False)
coordinates = list(records)
weights = weather_df['Humidity']
#customize map
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights = weights)

#assign marker layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#drop null values
vacation_df = weather_df.dropna(axis=0)
vacation_df

,City Name,Country Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
0,Severo-Kuril'sk,RU,50.68,156.12,51.37,83,100,5.08,2020-07-24 02:45:54
1,Lavrentiya,RU,65.58,-171.00,56.17,61,100,8.59,2020-07-24 02:49:45
2,San Patricio,MX,19.22,-104.70,82.40,74,75,4.83,2020-07-24 02:41:51
3,Barrow,US,71.29,-156.79,39.20,93,90,8.05,2020-07-24 02:45:25
4,Muisne,EC,0.60,-80.03,73.42,85,99,8.39,2020-07-24 02:49:45
...,...,...,...,...,...,...,...,...,...
575,Turukhansk,RU,65.82,87.98,56.21,70,100,7.96,2020-07-24 02:50:52
576,Sandpoint,US,48.28,-116.55,72.00,46,1,3.96,2020-07-24 02:50:11
577,Sancti Spíritus,CU,21.93,-79.44,74.57,91,100,3.44,2020-07-24 02:50:53
578,Ambilobe,MG,-13.20,49.05,68.68,61,78,1.61,2020-07-24 02:50:08


In [8]:
#select only ideal cities
#Criteria - Temperature >70 <85 degrees, cloudiness <50, humidity <60

ideal_cities_df = vacation_df.loc[(vacation_df["Temperature"] >=75) & 
                                  (vacation_df["Temperature"] <=80) & 
                                  (vacation_df["Cloudiness"] <50) & 
                                  (vacation_df["Humidity"] <50)]
ideal_cities_df

,City Name,Country Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Date
79,Tiznit Province,MA,29.58,-9.50,76.82,34,0,0.72,2020-07-24 02:49:53
110,Mount Isa,AU,-20.73,139.50,75.20,31,0,4.70,2020-07-24 02:47:09
115,Roebourne,AU,-20.78,117.13,77.23,33,0,12.82,2020-07-24 02:49:57
146,São Geraldo do Araguaia,BR,-6.40,-48.55,78.12,49,21,1.68,2020-07-24 02:50:01
200,Cabra,ES,37.47,-4.44,79.00,47,0,3.38,2020-07-24 02:50:08
233,Kalispell,US,48.20,-114.31,76.82,26,20,12.75,2020-07-24 02:50:11
301,Karratha,AU,-20.74,116.85,77.99,34,0,12.53,2020-07-24 02:46:08
457,Myaundzha,RU,63.05,147.19,79.23,37,30,6.73,2020-07-24 02:50:35
496,Laguna,US,38.42,-121.42,79.77,39,1,11.41,2020-07-24 02:50:41
515,Dayr Yūsuf,JO,32.49,35.80,75.20,41,0,8.05,2020-07-24 02:50:44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#create hotel df
hotel_df = ideal_cities_df.filter(["City Name", "Country Name", "Latitude", "Longitude"], axis=1)
#rename column headers to match requirements below for map
hotel_df.rename(columns={'City Name':'City',
                         'Country Name': 'Country',
                          'Latitude':'Lat',
                          'Longitude':'Lng'},inplace=True)
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lng,Hotel Name
79,Tiznit Province,MA,29.58,-9.50,
110,Mount Isa,AU,-20.73,139.50,
115,Roebourne,AU,-20.78,117.13,
146,São Geraldo do Araguaia,BR,-6.40,-48.55,
200,Cabra,ES,37.47,-4.44,
233,Kalispell,US,48.20,-114.31,
301,Karratha,AU,-20.74,116.85,
457,Myaundzha,RU,63.05,147.19,
496,Laguna,US,38.42,-121.42,
515,Dayr Yūsuf,JO,32.49,35.80,


In [12]:
#query from Google Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#loop through coordinates to get responses
for index,row in hotel_df.iterrows():

    
    #get location from df
    latitude = row["Lat"]
    longitude = row["Lng"]
    city = row["City"]
    
    #add to params
    locations = f"{latitude},{longitude}"
    
    #set up params
    params = {"location":locations,
               "keyword": "lodging",
                "radius":5000, 
                #"rankby":"distance",
                   "key": g_key,}
    
    #create URL for request
    response = requests.get(base_url, params=params).json()
    
    #extract results - print for testing
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #in case there are errors
    try:
        print("------------------------------------")
        print(f' Hotel found within 5000m of {city}')
        print("------------------------------------")
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("------------------------------------")
        print("Not result found... skipping.")
        print("------------------------------------")
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        

    

------------------------------------
 Hotel found within 5000m of Mount Isa
------------------------------------


C:\Users\Alexandra\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


------------------------------------
 Hotel found within 5000m of Roebourne
------------------------------------
------------------------------------
 Hotel found within 5000m of São Geraldo do Araguaia
------------------------------------
------------------------------------
 Hotel found within 5000m of Cabra
------------------------------------
------------------------------------
 Hotel found within 5000m of Kalispell
------------------------------------
------------------------------------
 Hotel found within 5000m of Karratha
------------------------------------
------------------------------------
 Hotel found within 5000m of Laguna
------------------------------------
------------------------------------
 Hotel found within 5000m of Dayr Yūsuf
------------------------------------
------------------------------------
 Hotel found within 5000m of Birjand
------------------------------------


In [13]:
#drop any with N/A value
#hotel_df = hotel_df[hotel_df["Hotel Name"] != "N/A"]
#view hotel names on df
hotel_df

,City,Country,Lat,Lng,Hotel Name
110,Mount Isa,AU,-20.73,139.50,Abacus Motel
115,Roebourne,AU,-20.78,117.13,Latitude20 Roebourne Village
146,São Geraldo do Araguaia,BR,-6.40,-48.55,Hotel Rio Araguaia
200,Cabra,ES,37.47,-4.44,Pensión Guerrero S.L.L.
233,Kalispell,US,48.20,-114.31,Kalispell Grand Hotel
301,Karratha,AU,-20.74,116.85,Latitude20 The Dunes Apartments
496,Laguna,US,38.42,-121.42,Hilton Garden Inn Sacramento Elk Grove
515,Dayr Yūsuf,JO,32.49,35.80,Sara Crown Hotel
541,Birjand,IR,32.87,59.22,زیبایی نارگل


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info,
    display_info_box=True)

# Add layer
fig.add_layer(marker_layer)

#update figure display
#gmaps.figure( zoom_level=16, center=(36.0999,80.2442)) #coordinates are for Winston Salem, NC. I wanted to default the zoom if I could.

# Display the map
fig


Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))